In [185]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows=8

## Read data from ornv-transmission.csv

In [186]:
df = pd.read_csv('ornv-transmission.csv', parse_dates=[3,4])

# Create a unique identifier for each beetle
df['btl_id'] = df.jar_id + df.sex

# Add column to flag duplicates in btl_id column
df['btl_id_dup'] = df.duplicated(subset='btl_id', keep=False)
df

,jar_id,sex,treatment,date_treated,date_died,postmortem_virus,postmortem_bacteria,btl_id,btl_id_dup
0,C1,m,none,2019-12-26,2020-01-16,NaN,NaN,C1m,False
1,C2,m,none,2019-12-26,2020-01-29,NaN,NaN,C2m,False
2,C3,m,none,2019-12-26,2020-01-19,NaN,NaN,C3m,False
3,C4,m,none,2019-12-26,2020-01-21,NaN,NaN,C4m,False
...,...,...,...,...,...,...,...,...,...
86,TM12,f,companion,2019-12-27,NaT,NaN,NaN,TM12f,False
87,TM13,f,companion,2019-12-27,2020-02-03,NaN,NaN,TM13f,False
88,TM14,f,companion,2019-12-27,2020-01-14,NaN,NaN,TM14f,False
89,TM15,f,companion,2019-12-27,2020-01-19,NaN,NaN,TM15f,False


In [187]:
# Check for duplicates in btl_id column; Halt processing if any found
if sum(df.btl_id_dup)>0:
    display(df[df.btl_id_dup])
    assert False, 'ERROR: There are rows in df which contain identical btl_id values.'

## Read data from pm-images/pm-images.csv

In [188]:
df_images = pd.read_csv('pm-images/pm-images.csv')

# Keep only rows only where 'unused' and 'duplicate' are not flagged (NaN)
df_images = df_images[pd.isna(df_images.unused) & pd.isna(df_images.duplicate)]

# Create a unique identifier for each beetle
df_images['btl_id'] = df_images.jar_id + df_images.sex

# Add column to flag duplicates in btl_id column
df_images['btl_id_dup'] = df_images.duplicated(subset='btl_id', keep=False)

df_images

,pm_image_filename,label,unused,duplicate,jar_id,sex,btl_id,btl_id_dup
0,20191229_120148.jpg,T-7 IF/UM,NaN,NaN,TF7,m,TF7m,False
1,20200102_111008.jpg,T-1 IM/UF male,NaN,NaN,TM1,m,TM1m,False
2,20200102_112130.jpg,C-7 female,NaN,NaN,C7,f,C7f,False
3,20200102_112827.jpg,C-7 male,NaN,NaN,C7,m,C7m,False
...,...,...,...,...,...,...,...,...
110,20200210_151921.jpg,C-13 male,NaN,NaN,C13,m,C13m,False
111,20200210_152419.jpg,C-13 female,NaN,NaN,C13,f,C13f,False
117,20200211_095032.jpg,C-12 male,NaN,NaN,C12,m,C12m,False
132,20200211_114845_001.jpg,T-2 IM/UF male,NaN,NaN,TM2,m,TM2m,False


In [189]:
# Check for duplicates in btl_id column; Halt processing if any found
if sum(df_images.btl_id_dup)>0:
    df_dup = df_images[df_images.btl_id_dup]
    df_dup = df_dup.sort_values(by='btl_id')
    display(df_dup)
    for i, r in df_dup.iterrows():
        image_filename = 'pm-images/{}'.format(r.pm_image_filename)
        print(image_filename, r.label)
        display(Image(image_filename))
    assert False, 'ERROR: There are rows in df_images which contain identical btl_id values.'

## Merge df and df_images and write to ornv-transition-1.csv

In [190]:
# Merge
df_merged = df.merge(df_images, on=['jar_id', 'sex'], how='left')
df_merged

,jar_id,sex,treatment,date_treated,date_died,postmortem_virus,postmortem_bacteria,btl_id_x,btl_id_dup_x,pm_image_filename,label,unused,duplicate,btl_id_y,btl_id_dup_y
0,C1,m,none,2019-12-26,2020-01-16,NaN,NaN,C1m,False,20200116_110145_001.jpg,C-1 male,NaN,NaN,C1m,False
1,C2,m,none,2019-12-26,2020-01-29,NaN,NaN,C2m,False,20200129_145244.jpg,C-2 male,NaN,NaN,C2m,False
2,C3,m,none,2019-12-26,2020-01-19,NaN,NaN,C3m,False,20200119_121343.jpg,C-3 male,NaN,NaN,C3m,False
3,C4,m,none,2019-12-26,2020-01-21,NaN,NaN,C4m,False,20200121_112302.jpg,C-4 male,NaN,NaN,C4m,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,TM12,f,companion,2019-12-27,NaT,NaN,NaN,TM12f,False,NaN,NaN,NaN,NaN,NaN,NaN
87,TM13,f,companion,2019-12-27,2020-02-03,NaN,NaN,TM13f,False,20200203_113806.jpg,T-13 IM/UF female,NaN,NaN,TM13f,False
88,TM14,f,companion,2019-12-27,2020-01-14,NaN,NaN,TM14f,False,20200114_104047.jpg,T-14 IM/UF female,NaN,NaN,TM14f,False
89,TM15,f,companion,2019-12-27,2020-01-19,NaN,NaN,TM15f,False,20200119_120121.jpg,T-15 IM/UF female,NaN,NaN,TM15f,False


In [191]:
# Drop unneeded columns
df_merged.drop(['label','unused','duplicate','btl_id_x','btl_id_dup_x',
                'btl_id_y','btl_id_dup_y'], axis=1, inplace=True)
df_merged

,jar_id,sex,treatment,date_treated,date_died,postmortem_virus,postmortem_bacteria,pm_image_filename
0,C1,m,none,2019-12-26,2020-01-16,NaN,NaN,20200116_110145_001.jpg
1,C2,m,none,2019-12-26,2020-01-29,NaN,NaN,20200129_145244.jpg
2,C3,m,none,2019-12-26,2020-01-19,NaN,NaN,20200119_121343.jpg
3,C4,m,none,2019-12-26,2020-01-21,NaN,NaN,20200121_112302.jpg
...,...,...,...,...,...,...,...,...
86,TM12,f,companion,2019-12-27,NaT,NaN,NaN,NaN
87,TM13,f,companion,2019-12-27,2020-02-03,NaN,NaN,20200203_113806.jpg
88,TM14,f,companion,2019-12-27,2020-01-14,NaN,NaN,20200114_104047.jpg
89,TM15,f,companion,2019-12-27,2020-01-19,NaN,NaN,20200119_120121.jpg


In [192]:
df_merged.to_csv('ornv-transmission-1.csv', index=False)